# Digital Twin

## What is constant product AMM?



In [5]:
import copy

genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.00},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
}

def swapToAsset2(state,inputs):
    agent = inputs[0]
    dA1 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA2 = state["AMM"]["r_2"]/(state["AMM"]["r_1"]+dA1*feeFactor)*dA1*feeFactor
    if dA1>0 and state[agent]["r_1"]-dA1 >= 0 :
        state["AMM"]["r_1"]+=dA1
        state[agent]["r_1"]-=dA1
        state["AMM"]["r_2"]-=dA2
        state[agent]["r_2"]+=dA2

def swapToAsset1(state,inputs):
    agent = inputs[0]
    dA2 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA1 = state["AMM"]["r_1"]/(state["AMM"]["r_2"]+dA2*feeFactor)*dA2*feeFactor
    if dA2>0 and state[agent]["r_2"]-dA2 >= 0 :
        state["AMM"]["r_2"]+=dA2
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]-=dA1
        state[agent]["r_1"]+=dA1

def addLiquidity(state,inputs):
    agent = inputs[0]
    R1=state["AMM"]["r_1"]
    R2=state["AMM"]["r_2"]
    S= state["AMM"]["s"]
    dA1=min(inputs[1],R1/R2*inputs[2])
    dA2=min(inputs[2],R2/R1*inputs[1])
    if (dA1 <= state[agent]["r_1"] and dA2 <= state[agent]["r_2"]) and (dA1 > 0 and dA2 > 0):
        state[agent]["r_1"]-=dA1
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]+=dA1
        state["AMM"]["r_2"]+=dA2
        dS = min(dA1/R1, dA2/R2) * S
        state["AMM"]["s"] += dS
        state[agent]["s"]+=dS

def removeLiquidity(state,inputs):
    dS = inputs[1]
    agent = inputs[0]
    if dS > 0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0:
        DR = (1-dS/state["AMM"]["s"])
        R1=state["AMM"]["r_1"]
        R2=state["AMM"]["r_2"]
        state[agent]["s"]-=dS
        state["AMM"]["r_1"]=R1*DR
        state["AMM"]["r_2"]=R2*DR
        state[agent]["r_1"]+=R1-state["AMM"]["r_1"]
        state[agent]["r_2"]+=R2-state["AMM"]["r_2"]
        state["AMM"]["s"]-=dS

def invariant(state):
    return state["AMM"]["r_1"] * state["AMM"]["r_2"]

def asset1(state):
    return state["AMM"]["r_1"]+state["Trader"]["r_1"]+state["LP"]["r_1"]

def asset2(state):
    return state["AMM"]["r_2"]+state["Trader"]["r_2"]+state["LP"]["r_2"]

def nice_print(self):
    if type(self)==float:
        return  round(self,3)
    if type(self)==int:
        return  self
    if type(self)==list:
        return  list(map(lambda l: print(l),self))
    if type(self)==dict:
        return  dict(map(lambda kv: (kv[0], print(kv[1])),self.items()))

def evolve(state, actionStack):
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

# test 1
# swapToAsset2(genesis,["Trader",10])
# genesis

# test 2
# swapToAsset2(genesis,["Trader",10])
# swapToAsset1(genesis,["Trader",9.0909090909091])
# genesis
actionList = [
        [ swapToAsset1 , [  genesis , ["Trader", 50] ]],
        [ swapToAsset2 , [  genesis , ["Trader", 25] ]]
]

evolve(genesis, actionList)




TypeError: can't multiply sequence by non-int of type 'float'